In [1]:
import os  
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2" 

import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D,GlobalAveragePooling2D, Dense, BatchNormalization, Activation
from keras.models import Model
from keras.layers import DepthwiseConv2D

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

num_classes        = 10
batch_size         = 64         # 64 or 32 or other
epochs             = 300
iterations         = 782       
USE_BN=True
DROPOUT=0.2 # keep 80%
CONCAT_AXIS=3
weight_decay=1e-4
DATA_FORMAT='channels_last' # Theano:'channels_first' Tensorflow:'channels_last'

log_filepath  = './mobilenet_slim_n'

Using TensorFlow backend.


In [2]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 200:
        return 0.001
    return 0.0001

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

In [3]:
def depthwise_separable(x,params):
    # f1/f2 filter size, s1 stride of conv
    (s1,f2) = params
    x = DepthwiseConv2D((3,3),strides=(s1[0],s1[0]), padding='same',
                        depthwise_initializer="he_normal",depthwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Conv2D(int(f2[0]), (1,1), strides=(1,1), padding='same',
               kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    return x

In [4]:
def MobileNet(img_input,shallow=False, classes=10):
    """Instantiates the MobileNet.Network has two hyper-parameters
        which are the width of network (controlled by alpha)
        and input size.
        # Arguments
            alpha: optional parameter of the network to change the 
                width of model.
            shallow: optional parameter for making network smaller.
            classes: optional number of classes to classify images
                into.
    """
    # change stride
    x = Conv2D(int(32), (3,3), strides=(1,1), padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = depthwise_separable(x,params=[(1,),(64,)])
    x = depthwise_separable(x,params=[(1,),(128,)])# change stride
    x = depthwise_separable(x,params=[(1,),(128,)])
    x = depthwise_separable(x,params=[(1,),(256,)])# change stride
    x = depthwise_separable(x,params=[(1,),(256,)])
    x = depthwise_separable(x,params=[(2,),(512,)])
    if not shallow:
        for _ in range(5):
            x = depthwise_separable(x,params=[(1,),(512,)])
            
    x = depthwise_separable(x,params=[(2,),(1024,)])
    x = depthwise_separable(x,params=[(1,),(1024,)])

    x = GlobalAveragePooling2D()(x)
    out = Dense(classes, activation='softmax')(x)
    return out

In [5]:
img_input=Input(shape=(32,32,3))
output = MobileNet(img_input)
model=Model(img_input,output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 32, 32, 32)        320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
__________

In [6]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('mobilenet_slim_n.h5')

Epoch 1/300
782/782 [==============================] - 105s 134ms/step - loss: 2.5651 - acc: 0.3934 - val_loss: 2.2788 - val_acc: 0.5124
Epoch 2/300
782/782 [==============================] - 91s 116ms/step - loss: 2.1312 - acc: 0.5647 - val_loss: 1.9730 - val_acc: 0.6253
Epoch 3/300
782/782 [==============================] - 92s 117ms/step - loss: 1.8514 - acc: 0.6635 - val_loss: 1.6894 - val_acc: 0.7235
Epoch 4/300
782/782 [==============================] - 91s 117ms/step - loss: 1.6569 - acc: 0.7277 - val_loss: 1.7267 - val_acc: 0.7154
Epoch 5/300
782/782 [==============================] - 90s 116ms/step - loss: 1.5251 - acc: 0.7704 - val_loss: 1.5727 - val_acc: 0.7599
Epoch 6/300
782/782 [==============================] - 91s 117ms/step - loss: 1.4245 - acc: 0.7977 - val_loss: 1.5189 - val_acc: 0.7819
Epoch 7/300
782/782 [==============================] - 92s 118ms/step - loss: 1.3416 - acc: 0.8231 - val_loss: 1.3903 - val_acc: 0.8120
Epoch 8/300
782/782 [==========================

Epoch 61/300
782/782 [==============================] - 91s 116ms/step - loss: 0.3960 - acc: 0.9733 - val_loss: 0.7998 - val_acc: 0.8883
Epoch 62/300
782/782 [==============================] - 91s 117ms/step - loss: 0.3939 - acc: 0.9740 - val_loss: 0.7312 - val_acc: 0.9049
Epoch 63/300
782/782 [==============================] - 92s 117ms/step - loss: 0.3800 - acc: 0.9770 - val_loss: 0.8034 - val_acc: 0.8916
Epoch 64/300
782/782 [==============================] - 91s 117ms/step - loss: 0.3771 - acc: 0.9766 - val_loss: 0.7832 - val_acc: 0.8887
Epoch 65/300
782/782 [==============================] - 91s 116ms/step - loss: 0.3764 - acc: 0.9770 - val_loss: 0.7644 - val_acc: 0.8964
Epoch 66/300
782/782 [==============================] - 91s 117ms/step - loss: 0.3663 - acc: 0.9779 - val_loss: 0.8162 - val_acc: 0.8851
Epoch 67/300
782/782 [==============================] - 92s 117ms/step - loss: 0.3716 - acc: 0.9750 - val_loss: 0.7025 - val_acc: 0.9056
Epoch 68/300
782/782 [===================

782/782 [==============================] - 91s 116ms/step - loss: 0.2135 - acc: 0.9996 - val_loss: 0.5522 - val_acc: 0.9295
Epoch 121/300
782/782 [==============================] - 91s 117ms/step - loss: 0.2126 - acc: 0.9996 - val_loss: 0.5461 - val_acc: 0.9307
Epoch 122/300
782/782 [==============================] - 91s 117ms/step - loss: 0.2118 - acc: 0.9996 - val_loss: 0.5589 - val_acc: 0.9290
Epoch 123/300
782/782 [==============================] - 91s 117ms/step - loss: 0.2113 - acc: 0.9995 - val_loss: 0.5552 - val_acc: 0.9289
Epoch 124/300
782/782 [==============================] - 91s 116ms/step - loss: 0.2104 - acc: 0.9995 - val_loss: 0.5516 - val_acc: 0.9293
Epoch 125/300
782/782 [==============================] - 91s 116ms/step - loss: 0.2098 - acc: 0.9995 - val_loss: 0.5581 - val_acc: 0.9283
Epoch 126/300
782/782 [==============================] - 91s 116ms/step - loss: 0.2086 - acc: 0.9998 - val_loss: 0.5590 - val_acc: 0.9290
Epoch 127/300
782/782 [=========================

782/782 [==============================] - 92s 117ms/step - loss: 0.1766 - acc: 0.9999 - val_loss: 0.5426 - val_acc: 0.9318
Epoch 179/300
782/782 [==============================] - 91s 116ms/step - loss: 0.1760 - acc: 0.9999 - val_loss: 0.5416 - val_acc: 0.9315
Epoch 180/300
782/782 [==============================] - 91s 116ms/step - loss: 0.1755 - acc: 0.9999 - val_loss: 0.5380 - val_acc: 0.9322
Epoch 181/300
782/782 [==============================] - 91s 117ms/step - loss: 0.1749 - acc: 0.9999 - val_loss: 0.5466 - val_acc: 0.9313
Epoch 182/300
782/782 [==============================] - 92s 118ms/step - loss: 0.1745 - acc: 0.9999 - val_loss: 0.5458 - val_acc: 0.9319
Epoch 183/300
782/782 [==============================] - 91s 117ms/step - loss: 0.1740 - acc: 0.9998 - val_loss: 0.5467 - val_acc: 0.9319
Epoch 184/300
782/782 [==============================] - 94s 120ms/step - loss: 0.1733 - acc: 0.9999 - val_loss: 0.5474 - val_acc: 0.9307
Epoch 185/300
782/782 [=========================

782/782 [==============================] - 92s 117ms/step - loss: 0.1625 - acc: 0.9999 - val_loss: 0.5310 - val_acc: 0.9322
Epoch 238/300
782/782 [==============================] - 92s 118ms/step - loss: 0.1627 - acc: 0.9999 - val_loss: 0.5289 - val_acc: 0.9328
Epoch 239/300
782/782 [==============================] - 93s 119ms/step - loss: 0.1625 - acc: 0.9999 - val_loss: 0.5318 - val_acc: 0.9314
Epoch 240/300
782/782 [==============================] - 93s 119ms/step - loss: 0.1624 - acc: 0.9999 - val_loss: 0.5334 - val_acc: 0.9317
Epoch 241/300
782/782 [==============================] - 93s 119ms/step - loss: 0.1623 - acc: 1.0000 - val_loss: 0.5375 - val_acc: 0.9320
Epoch 242/300
782/782 [==============================] - 93s 118ms/step - loss: 0.1623 - acc: 1.0000 - val_loss: 0.5337 - val_acc: 0.9324
Epoch 243/300
782/782 [==============================] - 92s 118ms/step - loss: 0.1622 - acc: 0.9999 - val_loss: 0.5282 - val_acc: 0.9334
Epoch 244/300
782/782 [=========================

782/782 [==============================] - 90s 115ms/step - loss: 0.1597 - acc: 0.9999 - val_loss: 0.5256 - val_acc: 0.9331
Epoch 297/300
782/782 [==============================] - 90s 115ms/step - loss: 0.1597 - acc: 0.9999 - val_loss: 0.5335 - val_acc: 0.9322
Epoch 298/300
782/782 [==============================] - 90s 115ms/step - loss: 0.1595 - acc: 1.0000 - val_loss: 0.5310 - val_acc: 0.9329
Epoch 299/300
782/782 [==============================] - 90s 115ms/step - loss: 0.1596 - acc: 0.9999 - val_loss: 0.5321 - val_acc: 0.9313
Epoch 300/300
782/782 [==============================] - 90s 115ms/step - loss: 0.1595 - acc: 0.9999 - val_loss: 0.5294 - val_acc: 0.9320
